# Selenium CGV 영화 리뷰 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 2.7MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## CGV 영화 리뷰 긁어오기

* 아이언맨: http://www.cgv.co.kr/movies/detail-view/?midx=38262#1
* 다크나이트: http://www.cgv.co.kr/movies/detail-view/?midx=76417#1
* url을 통해 리뷰 페이지 접근 불가
* 셀레니움으로 페이지 번호를 클릭하여 접근

In [22]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException


def get_movie_reviews(url, page_num = 10):

    wd = webdriver.Chrome('chromedriver', options = chrome_options)
    wd.get(url) 
    writer_list = []
    review_list = []
    date_list   = []

    for page_no in range(1, page_num + 1):
        try:

            page_ul = wd.find_element_by_id('paging_point')
            page_a  = page_ul.find_element_by_link_text(str(page_no))
            page_a.click()
            time.sleep(1)

            writers = wd.find_elements_by_class_name('writer-name')
            writer_list += [writer.text for writer in writers]
            reviews = wd.find_elements_by_class_name('box-comment')
            review_list += [review.text for review in reviews]
            dates = wd.find_elements_by_class_name('day')
            date_list += [date.text for date in dates]

            if page_no % 10 == 0 : 
                next_button = page_ul.find_element_by_class_name("btn-paging next")
                next_button.click() # 클릭
                time.sleep(1) # 클릭이후에는 1초 쉼
        except NoSuchElementException:
            break
            
    movie_review_df = pd.DataFrame({'Writer': writer_list,
                                   "Review": review_list,
                                   "Date": date_list})
    return movie_review_df


In [23]:
url = "http://www.cgv.co.kr/movies/detail-view/?midx=83327"
movie_review_df = get_movie_reviews(url, 12)
movie_review_df

,Writer,Review,Date
0,둥아,그 얼굴로 그것밖에 못 하나요,2020.08.13
1,ssiss,남자들의 영화에요~! 피가 좀 많이나옴..,2020.08.13
2,세니,"두번째봐요,,꿀잼 넘슬퍼용ㅜㅜ",2020.08.13
3,네멋,별로별로별로별로별로,2020.08.13
4,ao**nn21,액션이 너무 현실성없어요 히어로도 아니고 아무리세도 인간인데 주인공들이 때리면 맞은...,2020.08.13
5,빅보스0571,여보랑 본 네번째 영화 ??,2020.08.13
6,빅보스0571,여보랑 본 네번째 영화 ??,2020.08.13
7,wn**ud6039,"너무재밌었다~ 영화내내 눈을 뗄수가없었다,",2020.08.13
8,킴미ㅋ,오랜만에 재밌는 영화 즐감,2020.08.13
9,치로두리,오랜만에 재미있게 잘보고 갑니다.,2020.08.13


## CGV 상영작 스크래핑

* http://www.cgv.co.kr/movies/

In [27]:
url = "http://www.cgv.co.kr/movies/"

wd = webdriver.Chrome("chromedriver", options = chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name("sect-movie-chart")
contents = movie_chart.find_elements_by_class_name("box-contents")
for content in contents:
    link = content.find_element_by_tag_name("a").get_attribute('href')
    title = content.find_element_by_class_name("title").text
    percent = content.find_element_by_class_name('percent').text
    info = content.find_element_by_class_name('txt-info').text
    print(title, percent, info, link)
    print(get_movie_reviews(link, 2))


다만 악에서 구하소서 예매율30.2% 2020.08.05 개봉 http://www.cgv.co.kr/movies/detail-view/?midx=83327
         Writer                                             Review        Date
0       so**125                                        유민이좀 냅둬라!!!  2020.08.13
1     sw**63981                                     역시 믿고보는 배우들입니다  2020.08.13
2   re**ree0522                                    조금 잔인하긴 한데 재밌어요  2020.08.13
3       hs**mas                                     굿굿굿. 괜찮네요. 굿이요  2020.08.13
4     dm**y1324                            많은 어색한 설정이 몰입감을 떨어트렸습니더  2020.08.13
5      hh**12kr                                 마지막이 아쉬워요 그치먼 재밌네요  2020.08.13
6        Hyebbi                             좋았지만,, 결말이 너무 속상했어요,,ㅠ  2020.08.13
7      kt**ow12  너무 기대를 하고 봐서 그런지 모르겠지만 배우들의 연기들은 좋았지만 줄거리도 그렇고...  2020.08.13
8       rh**fma                                      재밌어요 긴장감있고 굿굿  2020.08.13
9          주홍돼지            역시 연기가 좋았어요. 솔직히 잔인한걸 잘 못 보지만 재밌게 봤습니다.  2020.08.13
10      UNICORN                    믿고 보는배우 두